In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import os
from Concurrent_Neural_Network.preprocessing import filter_index_from_dataframe, compute_proportion, add_temporal_features, dataframe_to_data_loader
from Concurrent_Neural_Network.models import Concurrent_Module
from Concurrent_Neural_Network.submodel import Multi_layer_feed_forward_model
import matplotlib.pyplot as plt

# Comparison 

Objectives: 

 - Compare graphicaly prevision from NN, Conc-NN, RF
 - Generate plot for articles
 
### I Data

#### I.A Read_Data

In [11]:
CUR_DIR = os.getcwd()
DATA_FILE = os.path.join(CUR_DIR,'data','smartphone_data.csv' )
data =  pd.read_csv(DATA_FILE, sep =';')
data.set_index(['product_id', 'monday_date'], inplace= True)

#### I.B Preproces_Data

In [12]:
minimal_sum=100 # A product must have at least minimal_sum to be considered
minimal_positive_length=15 # A product must have been sold during at least minimal_positive_length to be considered
horizon = 4 #Prediction horizon

In [34]:
filtered_data, products =  filter_index_from_dataframe(data, "sales", minimal_sum_target=minimal_sum, minimal_positive_length=minimal_positive_length )
market_shares, weekly_global_sales, products = compute_proportion(filtered_data, 'sales', resize=100) # Note that the proportion are explained in percentages
data_restrained = add_temporal_features(market_shares, horizon=horizon)
data_restrained['price'] = data_restrained['price'] /100
data_restrained.head()

<ipython-input-34-c3a8235426ba>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_restrained['price'] = data_restrained['price'] /100


sales  price    margin  sales_somme  proportion  \
product          monday_date                                                    
HIS6941785713652 2019-10-28    11.0   1.99  0.056784       6747.0    0.163035   
                 2019-11-04    13.0   1.99  0.076357       8775.0    0.148148   
                 2019-11-11     3.0   1.99  0.095930       6889.0    0.043548   
                 2019-11-18     2.0   1.99  0.095930       5225.0    0.038278   
                 2019-11-25     0.0   1.99  0.095930      27052.0    0.000000   

                              proportion_shift_4  proportion_shift_5  
product          monday_date                                          
HIS6941785713652 2019-10-28             0.020450            0.010707  
                 2019-11-04             0.031546            0.020450  
                 2019-11-11             0.023984            0.031546  
                 2019-11-18             0.036536            0.023984  
                 2019-11-25             0.163035            0.036536

#### I.C Training and Testing sets

In [35]:
date_learning =  '2020-06-01' # This date separate learning & testing set
date_stop = '2021_01_01' # This is the end of the testing set
# Split Train Test sets

X_train, X_test = data_restrained[data_restrained.index.get_level_values(1) < date_learning ], data_restrained[data_restrained.index.get_level_values(1) >= date_learning ]
X_test = X_test[X_test.index.get_level_values(1) < date_stop]

In [36]:
features = ['proportion_shift_4', 'proportion_shift_5','price', 'margin'] # Here are the features used for prediction
target ='proportion'
data_train_loader = dataframe_to_data_loader(X_train, features, target)
data_test_loader = dataframe_to_data_loader(X_test, features, target)

### II Training models

In this section, we will train three models :

 - A classical Neural Network with L1 Loss, named FF-NN
 - A concurrent Neural Network model with L1 Loss, named L1-Conc-NN
 - A concurrent Neural Network model with Poisson Loss, named Poisson-Conc-NN

 
All this models have the same architecture, describe by the list of numbers of cells

In [42]:
#parameters

n_input = len(features)
architecture = [6,3]
learning_rate_FF = 1e-4 # Learning rate for FF-NN models
learning_rate_Conc = 1e-4 # Learning rate for Conc-NN models
max_epochs = 1000
early_stopping = 50
batch_prt = (epochs // 10) 
sum_factor = 100

#### II.A Simple NN Network

A simple classical Feed-Forward Neural Network
NB ; Traning this models are complex, it may require to be repeted several times

In [43]:
FF_Model = Multi_layer_feed_forward_model(n_input, architecture, learning_rate=learning_rate_FF)
FF_Model.train(data_train_loader,eval_dataset=data_test_loader, max_epochs=max_epochs, batch_print=batch_prt)

Epoch 0
Train MAPE: 122.7741
Test MAPE: 104.4822 

Epoch 200
Train MAPE: 71.2243
Test MAPE: 69.8189 

Epoch 400
Train MAPE: 70.7054
Test MAPE: 69.2916 

Epoch 600
Train MAPE: 70.5162
Test MAPE: 69.0199 

Epoch 800
Train MAPE: 70.4311
Test MAPE: 68.7944 



### II.B Concurrent Neural Network with L1 Loss

In [ ]:
sub_NN_L1 = Multi_layer_feed_forward_model(n_input, architecture)
L1_Conc_NN_model = Concurrent_Module(sub_NN_L1, sum_factor=sum_factor, loss='L1', learning_rate=learning_rate_Conc)
L1_Conc_NN_model.train(data_train_loader,eval_dataset=data_test_loader, max_epochs=max_epochs, batch_print=batch_prt)

Epoch 0
Train MAPE: nan
Test MAPE: nan 

Epoch 200
Train MAPE: nan
Test MAPE: nan 

Epoch 400
Train MAPE: nan
Test MAPE: nan 

Epoch 600
Train MAPE: nan
Test MAPE: nan 



### II.C Concurrent Neural Network with Poisson Loss

In [ ]:
sub_NN_poisson = Multi_layer_feed_forward_model(n_input, architecture)
poisson_Conc_NN_model = Concurrent_Module(sub_NN_poisson , sum_factor=sum_factor, loss='poisson', learning_rate=learning_rate_Conc)
poisson_Conc_NN_model.train(data_train_loader,eval_dataset=data_test_loader, max_epochs=max_epochs, batch_print=batch_prt)